# 10.2 特徵：定義共同行為

> 特徵（trait）會定義特定型別與其他型別共享的功能。我們可以使用特徵定義來抽象出共同行為。我們可以使用特徵界限（trait bounds）來指定泛型型別為擁有特定行為的任意型別。

## 10.2.1 定義特徵

> 一個型別的行為包含我們對該型別可以呼叫的方法。
> 如果我們可以對不同型別呼叫相同的方法，這些型別就能定義共同行為了。
> 特徵定義是一個將方法簽名統整起來，來達成一些目的而定義一系列行為的方法。

基本上類似於其他語言中抽象型別的概念，將幾個行為類似重複的操作先規範起來，之後讓其他地方實作。

In [3]:
// 範例 10-12：Summary 特徵包含 summarize 方法所定義的行為
pub trait Summary {
    fn summarize(&self) -> String;
}

## 10.2.2 為型別實作特徵

- 這邊我們定義了一個 `Summary` 的特徵，並且在 `impl` 中實作了 `summary` 方法，這個方法會回傳一個字串。

In [4]:
// 範例 10-13：在型別 NewsArticle 與 Tweet 實作 Summary 特徵
pub struct NewsArticle {
    pub headline: String,
    pub location: String,
    pub author: String,
    pub content: String,
}

impl Summary for NewsArticle {
    fn summarize(&self) -> String {
        format!("{} {} 著 ({})", self.headline, self.author, self.location)
    }
}

pub struct Tweet {
    pub username: String,
    pub content: String,
    pub reply: bool,
    pub retweet: bool,
}

impl Summary for Tweet {
    fn summarize(&self) -> String {
        format!("{}: {}", self.username, self.content)
    }
}


In [5]:
fn main() {
    let tweet = Tweet {
        username: String::from("horse_ebooks"),
        content: String::from(
            "of course, as you probably already know, people",
        ),
        reply: false,
        retweet: false,
    };

    println!("帶了Summary: ");
    println!("1 則新推文：{}", tweet.summarize());
}

In [6]:
main()

帶了Summary: 
1 則新推文：horse_ebooks: of course, as you probably already know, people


()

- 這邊則是沒有定義 `Summary` 特徵的版本

雖然沒有也沒關係，但是這樣的程式相對就沒有比較明確的彙整

In [7]:
pub struct NewsArticleWithoutSummary {
    pub headline: String,
    pub location: String,
    pub author: String,
    pub content: String,
}

impl NewsArticleWithoutSummary {
    fn summarize(&self) -> String {
        format!("{} {} 著 ({})", self.headline, self.author, self.location)
    }
}

pub struct TweetWithoutSummary {
    pub username: String,
    pub content: String,
    pub reply: bool,
    pub retweet: bool,
}

impl TweetWithoutSummary {
    fn summarize(&self) -> String {
        format!("{}: {}", self.username, self.content)
    }
}


In [8]:
fn main() {
    let tweet = TweetWithoutSummary {
        username: String::from("horse_ebooks"),
        content: String::from(
            "of course, as you probably already know, people",
        ),
        reply: false,
        retweet: false,
    };

    println!("不用Summary定義特徵: ");
    println!("1 則新推文：{}", tweet.summarize());
}

In [9]:
main()

不用Summary定義特徵: 
1 則新推文：horse_ebooks: of course, as you probably already know, people


()

## 10.2.2 預設實作

> 有時候對特徵內的一些或所有方法定義預設行為是很實用的，而不必要求每個型別都實作所有方法。然後當我們對特定型別實作特徵時，我們可以保留或覆蓋每個方法的預設行為。

這個時候我們對於特徵的使用，就不再只是空有名號的抽象型別，而是真的可以有一些預設的實作，讓其他地方可以直接使用。

In [10]:
// 範例 10-14：Summary 特徵定義了 summarize 方法的預設實作
pub trait Summary {
    fn summarize(&self) -> String {
        String::from("(閱讀更多...)")
    }
}

pub struct NewsArticle {
    pub headline: String,
    pub location: String,
    pub author: String,
    pub content: String,
}

impl Summary for NewsArticle {}

這邊的 `NewArticle` 就是直接使用了預設的實作。

In [11]:
pub struct Tweet {
    pub username: String,
    pub content: String,
    pub reply: bool,
    pub retweet: bool,
}

impl Summary for Tweet {
    fn summarize(&self) -> String {
        format!("{}: {}", self.username, self.content)
    }
}


這邊的 `Tweet` 則在根據自己需要複寫了 `summary` 的實作。

In [12]:
fn main() {
    let article = NewsArticle {
        headline: String::from("Penguins win the Stanley Cup Championship!"),
        location: String::from("Pittsburgh, PA, USA"),
        author: String::from("Iceburgh"),
        content: String::from(
            "The Pittsburgh Penguins once again are the best \
             hockey team in the NHL.",
        ),
    };

    println!("有新文章發佈！{}", article.summarize());

    println!("");
    println!("----------------");
    println!("");

    let tweet = Tweet {
        username: String::from("馬力歐"),
        content: String::from("馬力歐賽車8 豪華版，將於 4 月 28 日登場！"),
        reply: false,
        retweet: false,
    };

    println!("1 則新推文：{}", tweet.summarize());
}

main()


有新文章發佈！(閱讀更多...)
----------------

1 則新推文：馬力歐: 馬力歐賽車8 豪華版，將於 4 月 28 日登場！


()

我們可以在特徵裡面定義一些，之後應用的他的實作必須被定義的東西

像下面我們現在給了 `Summary` 兩個方法， `summarzie` 和 `summarize_author`

`summarize` 有預設的樣式可以直接沿用，但是 `summarize_author` 則是必須被時作者自行實作。


In [13]:
pub trait SummaryNew {
    fn summarize_author(&self) -> String;

    fn summarize(&self) -> String {
        format!("(從 {} 閱讀更多...)", self.summarize_author())
    }
}

要使用這個版本的 `Summary`，我們只需要在對型別實作特徵時定義 `summarize_author` 就好

In [14]:
pub struct TweetNew {
    pub username: String,
    pub content: String,
    pub reply: bool,
    pub retweet: bool,
}

impl SummaryNew for TweetNew {
    fn summarize_author(&self) -> String {
        format!("@{}", self.username)
    }
}

In [15]:
fn main() {

    let tweet = TweetNew {
        username: String::from("馬力歐"),
        content: String::from("馬力歐賽車8 豪華版，將於 4 月 28 日登場！"),
        reply: false,
        retweet: false,
    };

    println!("1 則新推文：{}", tweet.summarize());
    println!("來自：{}", tweet.summarize_author());
}

main()

1 則新推文：(從 @馬力歐 閱讀更多...)
來自：@馬力歐


()

## 10.2.3 特徵作為參數

這個時候 `Summary` 便先發揮抽象型別的作用

In [16]:
pub fn notify(item: &impl Summary) {
    println!("頭條新聞！{}", item.summarize());
}

### a. 特徵界限語法

> impl Trait 語法看起來很直觀，不過它其實是一個更長格式的語法糖，這個格式稱之為「特徵界限（trait bound）」，它長得會像這樣

然後，我們再利用他去將我們的泛型限制在 `Summary` 這個特徵上，這樣編譯器即使還不知道他真正處理到的是什麼東西，但能先知道要做什麼，而不會報錯

In [17]:
pub fn notify<T: Summary>(item: &T) {
    println!("頭條新聞！{}", item.summarize());
}

> 此格式等同於之前段落的範例，只是比較長一點。我們將特徵界限置於泛型型別參數的宣告中，在尖括號內接在冒號之後。
>
> ```rust
> pub fn notify(item: &impl Summary) {
>     println!("頭條新聞！{}", item.summarize());
> }
> ```

- 實做多個變數時

```rust
pub fn notify(item1: &impl Summary, item2: &impl Summary) {
```
- 但如果我們想要兩個變數的型別都是一樣的話，可以這樣寫

```rust
pub fn notify<T: Summary>(item1: &T, item2: &T) {
```

### b. 透過 + 來指定多個特徵界限
如果這裡有好幾個特徵，我們可以透過 `+` 來指定多個特徵界限

```rust
pub fn notify(item: &(impl Summary + Display)) {
```

或

```rust
pub fn notify<T: Summary + Display>(item: &T) {
```

### c. 透過 where 來使特徵界限更清楚

> 用太多特徵界限也會帶來壞處。每個泛型都有自己的特徵界限，所以有數個泛型型別的函式可以在函式名稱與參數列表之間包含大量的特徵界限資訊，讓函式簽名難以閱讀。因此 Rust 有提供另一個在函式簽名之後指定特徵界限的語法 where。所以與其這樣寫：



```rust
fn some_function<T: Display + Clone, U: Clone + Debug>(t: &T, u: &U) -> i32 {
```

`=>`

```rust
fn some_function<T, U>(t: &T, u: &U) -> i32
where
    T: Display + Clone,
    U: Clone + Debug,
{
    ...
```

### d. 回傳有實作特徵的型別

> 我們也能在回傳的位置使用 impl Trait 語法來回傳某個有實作特徵的型別數值，如以下所示：

In [18]:
fn returns_summarizable() -> impl Summary {
    Tweet {
        username: String::from("horse_ebooks"),
        content: String::from(
            "of course, as you probably already know, people",
        ),
        reply: false,
        retweet: false,
    }
}
// returns_summarizable()

但你只能在回傳中使用**其中一種實作**，如果你的回傳會是會在多種實作中選其中一個則會出錯

<div>
    <img src="./does_not_compile.svg" width="100"/>
</div>

In [19]:
fn returns_summarizable(switch: bool) -> impl Summary {
    if switch {
        NewsArticle {
            headline: String::from(
                "Penguins win the Stanley Cup Championship!",
            ),
            location: String::from("Pittsburgh, PA, USA"),
            author: String::from("Iceburgh"),
            content: String::from(
                "The Pittsburgh Penguins once again are the best \
                 hockey team in the NHL.",
            ),
        }
    } else {
        Tweet {
            username: String::from("horse_ebooks"),
            content: String::from(
                "of course, as you probably already know, people",
            ),
            reply: false,
            retweet: false,
        }
    }
}


Error: `if` and `else` have incompatible types

> 寫說可能回傳 NewsArticle 或 Tweet 的話是不被允許的，因為 impl Trait 語法會限制在編譯器中最終決定的型別。我們會在第十七章的「允許不同型別數值的特徵物件」來討論如何寫出這種行為的函式。

---
#### f. 透過特徵界限來選擇性實作方法
> 在有使用泛型型別參數 impl 區塊中使用特徵界限，我們可以選擇性地對有實作特定特徵的型別來實作方法。
> 
> 舉例來說，下面的 `Pair<T>` 對所有 `T` 實作了 `new` 函式來回傳新的 `Pair<T>` 實例
> 
> （回想一下第五章的「定義方法」段落，`Self` 是 `impl` 區塊內的型別別名，在此例就是 `Pair<T>`）。
> 
> 但在下一個 `impl` 區塊中，只有在其內部型別 `T` 有實作能夠做比較的 `PartialOrd` 特徵以及能夠顯示在螢幕的 `Display` 特徵的話，才會實作 `cmp_display` 方法。

這就有點類似某些語言如Python中，透過 `overload` 來定義一個函數遭遇不同型別時的行為

In [ ]:
use std::fmt::Display;

struct Pair<T> {
    x: T,
    y: T,
}

// 當輸入隨便一個型別
// 就維持原樣回傳
impl<T> Pair<T> {
    // `Self` 是 `impl` 區塊內的型別別名，在此例就是 `Pair<T>`
    fn new(x: T, y: T) -> Self {
        Self { x, y }
    }
}

// 但如果是實作了 `Display` 與 `PartialOrd` 的型別
// 就定義一個新的 `cmp_display` 方法
impl<T: Display + PartialOrd> Pair<T> {
    // `Self` 是 `impl` 區塊內的型別別名，在此例就是 `Pair<T>`
    fn cmp_display(&self) {
        if self.x >= self.y {
            println!("最大的是 x = {}", self.x);
        } else {
            println!("最大的是 y = {}", self.y);
        }
    }
}
